# Pre-processing

In [75]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time

## URM

In [76]:
URM_all_dataframe = pd.read_csv("C:/Users/mauro/Downloads/Recommender System/Input/data_train.csv")
URM_all_dataframe.head(10)

,user_id,item_id,data
0,0,0,1.0
1,0,2,1.0
2,0,120,1.0
3,0,128,1.0
4,0,211,1.0
5,0,232,1.0
6,0,282,1.0
7,0,453,1.0
8,0,458,1.0
9,0,491,1.0


In [77]:
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("The number of interactions is {}".format(n_interactions))

Number of items	 38121, Number of users	 35736
Max ID items	 38120, Max Id users	 35735

The number of interactions is 1764607


## ICM

In [78]:
ICM_dataframe = pd.read_csv("C:/Users/mauro/Downloads/Recommender System/Input/data_ICM_metadata.csv")
ICM_dataframe.head(10)

,item_id,feature_id,data
0,2519,0,1.0
1,2642,0,1.0
2,3316,0,1.0
3,3400,0,1.0
4,3472,0,1.0
5,4361,0,1.0
6,4661,0,1.0
7,8697,0,1.0
8,9692,0,1.0
9,9693,0,1.0


In [79]:
featureID_unique = ICM_dataframe["feature_id"].unique()
itemID_unique = ICM_dataframe["item_id"].unique()

n_items = len(itemID_unique)
n_features = len(featureID_unique)
n_interactions = len(ICM_dataframe)

print ("Number of items\t {}, Number of features\t {}".format(n_items, n_features))
print ("Max ID items\t {}, Max Id features\t {}\n".format(max(itemID_unique), max(featureID_unique)))
print ("The number of combinations is {}".format(n_interactions))

Number of items	 38121, Number of features	 94331
Max ID items	 38120, Max Id features	 94330

The number of combinations is 2940040


## Construction of URM and ICM matrices

In [80]:
URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [81]:
ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [82]:
!git clone https://github.com/recsyspolimi/RecSys_Course_AT_PoliMi.git
%cd RecSys_Course_AT_PoliMi

c:\Users\mauro\Downloads\Recommender System\RecSys_Course_AT_PoliMi\RecSys_Course_AT_PoliMi\RecSys_Course_AT_PoliMi


Cloning into 'RecSys_Course_AT_PoliMi'...


In [83]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 177 ( 0.5%) Users that have less than 1 test interactions


In [84]:
import optuna
import pandas as pd
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function_KNN_similarities(optuna_trial):
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True     

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [85]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        new_row = pd.DataFrame([hyperparam_dict])
        
        if not new_row.empty:
            self.results_df = pd.concat([self.results_df, new_row], ignore_index=True)

In [86]:
# Start measuring time
start_time = time.time()

# Create study and optimize
optuna_study = optuna.create_study(direction="maximize")
save_results = SaveResults()
optuna_study.optimize(objective_function_KNN_similarities,
                      callbacks=[save_results],
                      n_trials=50)

# Calculate elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

# Convert the time to minutes or hours
hours = elapsed_time // 3600
minutes = (elapsed_time % 3600) // 60
seconds = elapsed_time % 60

print('\n')
# Display the time in a more readable format
if hours > 0:
    print(f"Training loop of 50 trials took {int(hours)} hours, {int(minutes)} minutes, and {int(seconds)} seconds.")
elif minutes > 0:
    print(f"Training loop of 50 trials took {int(minutes)} minutes and {int(seconds)} seconds.")
else:
    print(f"Training loop of 50 trials took {int(seconds)} seconds.")

[I 2024-11-10 19:03:42,358] A new study created in memory with name: no-name-a70169b4-9ec8-4046-a773-25dc0b59ae3f


Similarity column 38121 (100.0%), 3034.27 column/sec. Elapsed time 12.56 sec
EvaluatorHoldout: Processed 35559 (100.0%) in 42.29 sec. Users per second: 841


[I 2024-11-10 19:04:37,645] Trial 0 finished with value: 0.03618247919732636 and parameters: {'similarity': 'jaccard', 'topK': 108, 'shrink': 889}. Best is trial 0 with value: 0.03618247919732636.
C:\Users\mauro\AppData\Local\Temp\ipykernel_2768\3560955234.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results_df = pd.concat([self.results_df, new_row], ignore_index=True)


Similarity column 38121 (100.0%), 3189.30 column/sec. Elapsed time 11.95 sec
EvaluatorHoldout: Processed 35559 (100.0%) in 56.26 sec. Users per second: 632


[I 2024-11-10 19:05:47,861] Trial 1 finished with value: 0.031666938961715574 and parameters: {'similarity': 'jaccard', 'topK': 778, 'shrink': 846}. Best is trial 0 with value: 0.03618247919732636.


...

Similarity column 38121 (100.0%), 3412.47 column/sec. Elapsed time 11.17 sec
EvaluatorHoldout: Processed 35559 (100.0%) in 38.78 sec. Users per second: 917


[I 2024-11-10 19:48:16,870] Trial 48 finished with value: 0.040652519376469465 and parameters: {'similarity': 'jaccard', 'topK': 438, 'shrink': 121}. Best is trial 41 with value: 0.044327081930007714.


Similarity column 38121 (100.0%), 3468.34 column/sec. Elapsed time 10.99 sec
EvaluatorHoldout: Processed 35559 (100.0%) in 36.51 sec. Users per second: 974


[I 2024-11-10 19:49:05,258] Trial 49 finished with value: 0.04312164290334188 and parameters: {'similarity': 'jaccard', 'topK': 378, 'shrink': 41}. Best is trial 41 with value: 0.044327081930007714.




Training loop of 50 trials took 45 minutes and 22 seconds.


In [87]:
pruned_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study.best_trial.value)

Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  0
  Number of complete trials:  50
Best trial:
  Value Validation:  0.044327081930007714


In [88]:
optuna_study.best_trial

FrozenTrial(number=41, state=TrialState.COMPLETE, values=[0.044327081930007714], datetime_start=datetime.datetime(2024, 11, 10, 19, 41, 49, 770714), datetime_complete=datetime.datetime(2024, 11, 10, 19, 42, 34, 276324), params={'similarity': 'jaccard', 'topK': 231, 'shrink': 21}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'similarity': CategoricalDistribution(choices=('cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean')), 'topK': IntDistribution(high=1500, log=False, low=0, step=1), 'shrink': IntDistribution(high=1000, log=False, low=0, step=1)}, trial_id=41, value=None)

In [89]:
optuna_study.best_trial.params

{'similarity': 'jaccard', 'topK': 231, 'shrink': 21}

In [90]:
save_results.results_df

,result,similarity,topK,shrink,tversky_alpha,tversky_beta,asymmetric_alpha,normalize_avg_row,similarity_from_distance_mode,normalize
0,0.036182,jaccard,108.0,889.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.031667,jaccard,778.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.032099,tversky,1466.0,515.0,1.011369,0.362473,NaN,NaN,NaN,NaN
3,0.037189,tversky,956.0,246.0,1.197593,0.732883,NaN,NaN,NaN,NaN
4,0.035447,asymmetric,270.0,187.0,NaN,NaN,0.307497,NaN,NaN,NaN
5,0.034764,tversky,357.0,916.0,1.804853,0.069819,NaN,NaN,NaN,NaN
6,0.032817,asymmetric,1291.0,620.0,NaN,NaN,1.344330,NaN,NaN,NaN
7,0.014677,euclidean,1326.0,76.0,NaN,NaN,NaN,True,lin,True
8,0.033480,asymmetric,632.0,864.0,NaN,NaN,1.798765,NaN,NaN,NaN
9,0.038445,tversky,18.0,879.0,0.280533,1.270194,NaN,NaN,NaN,NaN


In [91]:
recommender_instance = ItemKNNCFRecommender(URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

Similarity column 38121 (100.0%), 3220.71 column/sec. Elapsed time 11.84 sec


# Testing

In [92]:
data_target_users_test = pd.read_csv("C:/Users/mauro/Downloads/Recommender System/Input/data_target_users_test.csv")
data_target_users_test.head(10)

,user_id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [ ]:
target_result = []
for target in data_target_users_test["user_id"]:
    target_result.append(recommender_instance.recommend(target, cutoff = 10, remove_seen_flag=True))

In [ ]:
user_ids = data_target_users_test["user_id"]

# Convert `target_result` to the required format for each user
formatted_data = {
    "user_id": user_ids,
    "item_list": [" ".join(map(str, items)) for items in target_result]
}

# Create the DataFrame
submission_df = pd.DataFrame(formatted_data)

submission_df.to_csv("C:/Users/mauro/Downloads/Recommender System/Submissions/submission_optuna_ItemKNNCFRecommender.csv", index=False, header=["user_id", "item_list"])